In [ ]:
# EV Routing pakcage
from ev_routing import EVRouting

# Visualization packages
import plotly.graph_objects as go

import os


# Initializing EVRouting
bl = { 'lat': 52.50e0, 'lon': 13.34e0 } # Bottom left corner coordinate
tr = { 'lat': 52.53e0, 'lon': 13.43e0 } # Top right corner coordinate

area = [ bl['lat'], bl['lon'], tr['lat'], tr['lon'] ]
evr = EVRouting( area )


# Initializing bapbox
mapbox_access_token = os.environ['mapbox_access_token']

latitudes = [ v['lat'] for v in evr.v.values() ]
longitudes = [ v['lon'] for v in evr.v.values() ]

node_ids = [ v['id'] for v in evr.v.values() ]
node_labels = [ str(i) for i in range(len(evr.v)) ]

In [ ]:
nodes_map = go.Figure()

nodes_map.add_trace( go.Scattermapbox(
    lat = latitudes,
    lon = longitudes,
    mode = 'markers',
    marker = go.scattermapbox.Marker(
        size = 6,
        color = 'rgb(242, 177, 172)',
        opacity=0.7
    ),
    text = node_labels,
    hoverinfo = 'text'
))

nodes_map.update_layout(
    autosize = True,
    hovermode = 'closest',
    showlegend = False,
    mapbox = go.layout.Mapbox(
        accesstoken = mapbox_access_token,
        bearing = 0,
        center = go.layout.mapbox.Center(
            lat = evr.map_center[0],
            lon = evr.map_center[1]
        ),
        pitch = 0,
        zoom = 12,
        style = 'light'
    ),
)

nodes_map.update_layout(margin = { "r": 0, "t": 0, "l": 0, "b": 0 })
nodes_map.show()

In [ ]:
s = node_ids[2388]
t = node_ids[10439]
bs = 1e6

bt, SoC, trace = evr.dijkstra( s, t, bs )
bt

In [ ]:
trace_resolution = 50
SoC_resolution = ( 30, 15 )


steps = int( len(trace) / 100 ) + 1

x_px = ( tr['lon'] - bl['lon'] ) / SoC_resolution[0]
y_px = ( tr['lat'] - bl['lat'] ) / SoC_resolution[1]

regions = { 'type': 'FeatureCollection', 'features': [] }
for i in range( SoC_resolution[0] ):
    for j in range( SoC_resolution[1] ):
        key = int( .5 * (i + j) * (i + j + 1) + j )
        regions['features'].insert(0, {
            'id': key, 'type': 'Feature', 'properties': {}, 'geometry': {
                'type': 'Polygon', 'coordinates': [[
                    [ i     * x_px + bl['lon'], j     * y_px + bl['lat'] ],
                    [ (i+1) * x_px + bl['lon'], j     * y_px + bl['lat'] ],
                    [ (i+1) * x_px + bl['lon'], (j+1) * y_px + bl['lat'] ],
                    [ i     * x_px + bl['lon'], (j+1) * y_px + bl['lat'] ],
                    [ i     * x_px + bl['lon'], j     * y_px + bl['lat'] ]
                ]]
            }
        })


z_dict = {}
for soc in SoC.values():
    if soc['prev'] == -1: continue
    
    i = int( ( evr.v[ soc['prev'] ]['lon'] - bl['lon'] ) / x_px )
    j = int( ( evr.v[ soc['prev'] ]['lat'] - bl['lat'] ) / y_px )
    key = int( .5 * (i + j) * (i + j + 1) + j )
    
    z_dict[key] = z_dict[key] + 1 if key in z_dict else 1
    

locations = []
z = []
for i in range( SoC_resolution[0] ):
    for j in range( SoC_resolution[1] ):
        key = int( .5 * (i + j) * (i + j + 1) + j )
        locations.insert(0, key)
        if key in z_dict:
            z.insert(0, z_dict[key])
        else:
            z.insert(0, 0)


trace_map = go.Figure(go.Choroplethmapbox(
    geojson = regions,
    locations = locations,
    z = z,
    colorscale = 'MAGMA', # 'RdGy',
    reversescale = False, # True,
    marker_opacity = 0.4,
    marker_line_width = 0
))


pv = evr.v[s]
for vid in trace[ ::steps ]:
    lat = [ pv['lat'], evr.v[vid]['lat'] ]
    lon = [ pv['lon'], evr.v[vid]['lon'] ]
    pv = evr.v[vid]

    
    trace_map.add_trace( go.Scattermapbox(
        lat = lat,
        lon = lon,
        mode = 'lines',
        line = go.scattermapbox.Line(
            width = 6,
            color = 'rgb(87, 160, 211)'
        )
    ))
    
trace_map.add_trace( go.Scattermapbox(
    lat = [ evr.v[s]['lat'], evr.v[t]['lat'] ],
    lon = [ evr.v[s]['lon'], evr.v[t]['lon'] ],
    mode = 'markers',
    marker = go.scattermapbox.Marker(
        size = 10,
        color = 'rgb(255, 255, 255)',
    ),
    text = [ 'Start', 'End' ],
    hoverinfo = 'text'
))



trace_map.update_layout(
    autosize = True,
    hovermode = 'closest',
    showlegend = False,
    mapbox = go.layout.Mapbox(
        accesstoken = mapbox_access_token,
        bearing = 0,
        center = go.layout.mapbox.Center(
            lat = evr.map_center[0],
            lon = evr.map_center[1]
        ),
        pitch = 0,
        zoom = 12,
        style = 'light'
    ),
)

trace_map.update_layout(margin = { "r": 0, "t": 0, "l": 0, "b": 0 })
trace_map.show()